In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import resample
import re

In [3]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:

import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import gensim

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/SemEval2016-Task6-raw-annotations-stance.csv')
print(df.head)

<bound method NDFrame.head of         Worker ID  Instance ID                    Target  \
0        worker_1       2370.0  Legalization of Abortion   
1        worker_2       2370.0  Legalization of Abortion   
2        worker_3       2370.0  Legalization of Abortion   
3        worker_4       2370.0  Legalization of Abortion   
4        worker_5       2370.0  Legalization of Abortion   
...           ...          ...                       ...   
53094  worker_113      10951.0           Hillary Clinton   
53095   worker_85      10951.0           Hillary Clinton   
53096   worker_56      10951.0           Hillary Clinton   
53097   worker_84      10951.0           Hillary Clinton   
53098   worker_79      10951.0           Hillary Clinton   

                                                   Tweet   Stance  \
0      Thank you for another day of life Lord. #Chris...  AGAINST   
1      Thank you for another day of life Lord. #Chris...    FAVOR   
2      Thank you for another day of life L

In [6]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['Stance'] = encoder.fit_transform(df['Stance'])

In [7]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=4391719 sha256=766c92702429d9edd5de4403b23bc725a96edfc4fd751d73d4f42b13f230ab06
  Stored in directory: /root/.cache/pip/wheels/64/57/bc/1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext


In [7]:
import fasttext

In [8]:
model=fasttext.load_model('/content/drive/MyDrive/Twitter-semeval2016-master/Twitter-semeval2016-master/random_partitions_of_unlabeled_corpus/cc.en.300.bin')


In [9]:
def rnn_fasttext(df,model):


    # Separate the majority and minority classes
    df_majority = df[df['Stance'] == 0]
    df_minority = df[df['Stance'] == 1]

    # Resample the majority class to match the number of samples in the minority class
    df_majority_resampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)

    # Concatenate the resampled majority class with the minority class
    df_resampled = pd.concat([df_majority_resampled, df_minority])

    # Shuffle the rows of the resampled dataset
    df_resampled = df_resampled.sample(frac=1, random_state=42)
    df = df_resampled

    def clean_data(review):
        review = re.sub('[^a-zA-Z]', ' ', review)
        review = review.lower()
        return review

    df['Tweet'] = df['Tweet'].apply(clean_data)

    def remove_stop_words(review):
        review_minus_sw = []
        stop_words = stopwords.words('english')
        review = review.split()
        review = [review_minus_sw.append(word) for word in review if word not in stop_words]
        review = ' '.join(review_minus_sw)
        return review

    df['Tweet'] = df['Tweet'].apply(remove_stop_words)

    def lematize(review):
        lemmatizer = WordNetLemmatizer()
        review = review.split()
        review = [lemmatizer.lemmatize(word) for word in review]
        review = ' '.join(review)
        return review

    df['Tweet'] = df['Tweet'].apply(lematize)

    message = df['Tweet']
    stance = df['Stance']
    X_train_embeddings = np.array([model.get_word_vector(x) for x in message])
    #input_data = pad_sequences(word_emd, maxlen=100, padding='post')
    #train_embeds = [[0] * 300] * len(message)
    #for idx, tweet in enumerate(message):
    #  for word in tweet:
    #    train_embeds[idx] += model[word]
    #train_embeds=np.array(train_embeds)
    X_train, X_test , y_train, y_test = train_test_split(X_train_embeddings, stance, test_size=0.20)


    max_length = 300
    vocab_size = 10000

    def define_rnn_model():
        model = Sequential()
        model.add(Embedding(vocab_size, 300, input_length=max_length))
        model.add(LSTM(64))
        model.add(Dense(1, activation='relu'))
        return model

    # call the define_rnn_model function to create the model object
    model = define_rnn_model()

    # print the model summary
    model.summary()

    # calling adam optimizer
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # visualizing each eppoch
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

    # caluclating loss and accuracy
    loss, accuracy = model.evaluate(X_test,y_test)

    # printing accuracy
    print('Testing Accuracy is {} '.format(accuracy*100))

    y_predict = (model.predict(X_test)>=0.5).astype(int)
    from sklearn.metrics import classification_report
    print(classification_report(y_test,y_predict,target_names = ["Against","Favor"]))


In [10]:
print("*****************Feminist Movement******************")
rnn_fasttext(df[df['Target'] == 'Feminist Movement'],model)

*****************Feminist Movement******************
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          3000000   
                                                                 
 lstm (LSTM)                 (None, 64)                93440     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 3,093,505
Trainable params: 3,093,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
177/177 [==============================] - 98s 533ms/step - loss: 0.7115 - accuracy: 0.4770 - val_loss: 0.6967 - val_accuracy: 0.4929
Epoch 2/10
177/177 [==============================] - 94s 531ms/step - loss: 0.6952 - accuracy: 0.4940 - val_los

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
#for multiple targets
for i in df["Target"].unique():
  tar = df[df["Target"]==i]
  print("*******************",i,"*********************")
  rnn_fasttext(tar,model)

In [ ]:
#for total data set
rnn_fasttext(df,model)